## MP2: Red & White Wine

In [ ]:
!ollama pull llama3.2-vision:11b

In [ ]:
!ollama list

In [ ]:
!pip install ollama

In [ ]:
!pip install ollama-ocr

In [ ]:
import os

In [ ]:
import pandas as pd, numpy as np, seaborn as sbn, matplotlib.pyplot as plt

from descriptive_stats import ct, dp, cr, vs
from utils.data_loader import load_excel, merge_wine_data, remove_missing, remove_duplicates
from utils.read_image import explain
from utils.read_api import readAPI
from scipy.stats import zscore
from IPython.display import Image, display, Markdown

### Data Loading

##### 1. Load wine data into python dataframes

In [ ]:
media_path = '../Media'

In [ ]:
xlsx_file_path = '../Data/winequality-red.xlsx'
df_red = load_excel(xlsx_file_path)

In [ ]:
xlsx_file_path = '../Data/winequality-white.xlsx'
df_white = load_excel(xlsx_file_path)

In [ ]:
df_red

In [ ]:
df_white

##### 2. Clean the data

In [ ]:
df_red.isna().sum()

In [ ]:
remove_missing(df_red)

In [ ]:
df_white.isna().sum()

In [ ]:
remove_missing(df_white)

##### 3. Aggregate the two sources into one, keeping the identity of each type "red"/"white". merge_wine_data function creates type column for this purpose.

In [ ]:
wine_df = merge_wine_data(df_red, df_white)

In [ ]:
wine_df

### Data Exploration

##### 4. Explore the features of each dataframe seperately. Identify independent and dependent variables.

In [ ]:
print("Red Wines:")
df_red.info()

print("\nWhite Wines:")
df_white.info()

print("\nCombined data:")
wine_df.info()

In [ ]:
wine_df.describe()

##### Judging from the output Quality is a dependent variable with the rest being independent.

### 5.
Transform the categorical data into numeric, applying appropriate encoding methods.

In [ ]:

wine_df['type'] = wine_df['type'].map({'red': 0, 'white': 1})

In [ ]:
wine_df

### 6.
Calculate the descriptive statistics of the numeric data. Check whether the distribution of
the values of the attributes is normal.

In [ ]:
# Descriptive statistics - not sure if this is what is ment?
print(wine_df.describe())

In [ ]:
#smaller scale histogram
wine_df.hist(bins=15, figsize=(15, 10))
plt.tight_layout()
plt.show() 

In [ ]:
# Huge scaling, but otherwise i cant see the numbers
vs.histogram(wine_df, title='Wine Features Histogram')

In [ ]:
#tells you if the distribution is asymmetric. 
#A value of 0 means perfectly symmetric, positive skewness means the right tail is longer, and negative skewness means the left tail is longer.
wine_df.skew()

In [ ]:
# tells you how heavy or light the tails of the distribution are compared to a normal distribution.
# A kurtosis greater than 3 indicates heavy tails (more outliers), while less than 3 means lighter tails.
wine_df.kurtosis()

### 7. Diagrams
Plot diagrams that visualize the differences in red and white wine samples

In [ ]:
df_red_clean = remove_duplicates(df_red)

In [ ]:
df_white_clean = remove_duplicates(df_white)

In [ ]:
wine_df_clean = remove_duplicates(wine_df)

In [ ]:
vs.countplot_comparison(
    df_list=[df_red_clean, df_white_clean, wine_df_clean],
    labels=['Red Wine', 'White Wine', 'Combined'],
    category_col='quality',
    hue_col='type',
    title_prefix='Quality Distribution -'
)

In [ ]:
# Which type of wine has higher average quality?
mean_quality_red = wine_df_clean[wine_df_clean['type'] == 0]['quality'].mean()
mean_quality_white = wine_df_clean[wine_df_clean['type'] == 1]['quality'].mean()
difference = mean_quality_white - mean_quality_red
print(f"Red: {mean_quality_red:.2f}, White: {mean_quality_white:.2f}, Difference: {difference:.2f}")

In [ ]:
# Remember type 0 is Red and type 1 is White
vs.barplot(wine_df_clean, x_col='type', y_col='quality', title='Average Wine Quality by Type', ylabel='Average Quality')

##### We now see that, once cleaned of missing values AND duplicates, white wines have a slightly higher average quality by 0.23

In [ ]:
avg_alcohol = wine_df_clean.groupby('type')['alcohol'].mean()
print(avg_alcohol)

In [ ]:
vs.boxplot(data=wine_df_clean, x='type', y='alcohol', title='Alcohol Content by Wine Type')

##### We can see that white wine has a higher average alchohol content with 10.5893 vs red wine at 10.4323, with a few small outliers for red wine

In [ ]:
upper_lim = wine_df_clean['alcohol'].quantile(.95)

In [ ]:
wine_df_no_outliers = wine_df_clean[(wine_df_clean['alcohol'] < upper_lim)]

In [ ]:
avg_alcohol_new = wine_df_no_outliers.groupby('type')['alcohol'].mean()
print(avg_alcohol_new)

In [ ]:
vs.boxplot(data=wine_df_no_outliers, x='type', y='alcohol', title='Alcohol Content by Wine Type')

##### Now with all outliers removed above the 95th percentile

In [ ]:
avg_sugar = wine_df_clean.groupby('type')['residual sugar'].mean()
print(avg_sugar)

In [ ]:
vs.boxplot(data=wine_df_clean, x='type', y='residual sugar', title='Residual Sugar by Wine Type')

##### We see that white wine a significantly higher average residual sugar content at 5.9148 compared to red wine at 2.5234.
##### However, we also see from the boxplot that there is one massive outlier from the white wine datasheet

In [ ]:
# Let's find this one obvious mega outlier
white_wine = wine_df_clean[wine_df_clean['type'] == 1].copy()
max_sugar_index = white_wine['residual sugar'].idxmax()

# Let's see the outlier itself
outlier_row = wine_df_clean.loc[max_sugar_index]
print("Outlier row with highest residual sugar (white wine):")
print(outlier_row)

In [ ]:
wine_df_outlier_gone = wine_df_clean.drop(index=max_sugar_index)

In [ ]:
# Now let's try again
avg_sugar = wine_df_outlier_gone.groupby('type')['residual sugar'].mean()
print(avg_sugar)

In [ ]:
vs.boxplot(data=wine_df_outlier_gone, x='type', y='residual sugar', title='Residual Sugar by Wine Type')

##### We see that white wine still has a significantly higher average residual sugar content at 5.8996 compared to red wine at 2.5234.
##### The biggest obvious outlier has been removed, yet other outliers remain from both types of wine. Let's try and remove these now. 

In [ ]:
r_wine = wine_df_clean[wine_df_clean['type'] == 0]
r_outliers = dp.iqr_outliers(r_wine, 'residual sugar')
w_wine = wine_df_clean[wine_df_clean['type'] == 1]
w_outliers = dp.iqr_outliers(w_wine, 'residual sugar')

In [ ]:
all_wine_outliers = pd.concat([r_outliers, w_outliers])

In [ ]:
# Now we remove the outliers
wine_df_outlier_free = wine_df_clean.drop(index=all_wine_outliers.index)

In [ ]:
avg_sugar = wine_df_outlier_free.groupby('type')['residual sugar'].mean()
print(avg_sugar)

In [ ]:
vs.boxplot(data=wine_df_outlier_free, x='type', y='residual sugar', title='Residual Sugar by Wine Type')

##### The boxplot is now clearer. Using IQR we still see some small outliers just beyond the "whiskers", this is likely since we've removed outliers for red and white wines seperately before merging the cleaned data back together. The new boxplot has likely calculated new IQR Q3 and Q1 values meaning that new outliers appear. We could potentially remove them, but given that they are barely even outside the "whiskers" they don't really distort our analysis. Maybe these could be seen as a form of natural variation?

In [ ]:
# Does the quantity of alcohol and residual sugar influence the quality of the wine?
vs.lmplot(wine_df_outlier_gone, x='alcohol', y='quality', hue='type', title='Alcohol vs Quality')

vs.lmplot(wine_df_outlier_gone, x='residual sugar', y='quality', hue='type', title='Residual Sugar vs Quality')

##### These implots plot all the wines allowing us to see quality and compare these to either residual sugar or alchohol. The regression line indicates a trend and thus also gives an impression of whether there might be a positive or negative correlation

In [ ]:
red_corr = wine_df_outlier_gone[wine_df_outlier_gone['type'] == 0][['alcohol', 'residual sugar', 'quality']].corr()
white_corr = wine_df_outlier_gone[wine_df_outlier_gone['type'] == 1][['alcohol', 'residual sugar', 'quality']].corr()
print("Red Wine Correlation:\n", red_corr)
print("White Wine Correlation:\n", white_corr)

In [ ]:
# A correlation heatmap for the entire dataframe containing both red and white wines
vs.correlation_heatmap(wine_df_outlier_gone)

##### We see that there's a positive correlation between alchohol content and quality for both red and white wine in our scatterplots, with it being only mildly more positive for red wine compared to white.
##### The correlation between residual sugar and quality however is not quite as apparent. White wine appears to have a slightly negative correlation while red wine looks barely positive BUT for red wine the correlation is uncertain enough that we can make a final conclusion based solely on diagrams.
##### However judging from the Pearson Correlation Matrices, knowing that 1 means a strong posititive correlation, -1 a strong negative and 0 no correlation, we can arrive at the following conclusion:

##### For red wine, with alchohol vs. quality correlation at 0.48, we have a moderate positive correlation - meaning more alchohol = better quality. Meanwhile with residual sugar vs. alchohol at only 0.014, there is pratically no correlation here.
##### For white wine, with alchohol vs. quality correlation at 0.46, we have a moderate positive correlation - meaning again more alchohol = better quality. However when it comes to residual sugar vs. quality at -0.12, we have a weak negative correlation - meaning quality drops slightly for white wines the more sugar is in it.

### 8. Questions of interest
Discuss which other questions might be of interest for wine consumers and distributors

In [ ]:
# Task 8: Discuss which other questions might be of interest for wine consumers and distributors

print("Questions of interest for wine consumers:")
print("1. Is there a relationship between price (if available) and quality?")
print("2. Which attributes most affect the taste profile that consumers prefer?")
print("3. Are there significant differences between red and white wines in terms of health attributes like alcohol content?")
print("4. Which wines offer the best quality-to-price ratio?")
print("5. Are organic/sustainable wines (if labeled) rated differently than conventional wines?")

print("\nQuestions of interest for wine distributors:")
print("1. Which chemical properties predict the shelf life and stability of wines?")
print("2. Is there a correlation between production cost factors and quality?")
print("3. Which attributes should be highlighted in marketing to different consumer segments?")
print("4. Are there seasonal variations in wine quality or preferences?")
print("5. How do transportation and storage conditions affect wine attributes and quality?")


### 9. Split data aggregated data into subsets
Split the aggregated data into five subsets by binning the attribute pH. Which subset has 
highest density? What if you split the data in ten subsets?

In [ ]:
# Task 9: Split data into subsets by binning the pH attribute

# Define pH bins and labels
ph_bins = [2.7, 3.1, 3.3, 3.5, 3.7, 4.0]  # Adjust these ranges based on your data distribution
ph_labels = ['Very high acidity', 'High acidity', 'Medium acidity', 'Low acidity', 'Very low acidity']

# Create a new column with pH bins
wine_df['pH_category'] = pd.cut(wine_df['pH'], bins=ph_bins, labels=ph_labels)

# Display the distribution of wines across pH categories
ph_distribution = wine_df['pH_category'].value_counts().sort_index()
print("\nDistribution of wines across pH categories:")
print(ph_distribution)

# Create a bar plot of the pH distribution
plt.figure(figsize=(10, 6))
ph_distribution.plot(kind='bar', color='skyblue')
plt.title('Distribution of Wines by pH Level', fontsize=14)
plt.xlabel('pH Category', fontsize=12)
plt.ylabel('Number of Wines', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Calculate mean quality for each pH category
quality_by_ph = wine_df.groupby('pH_category')['quality'].mean().sort_index()
print("\nMean quality by pH category:")
print(quality_by_ph)

# Create a bar plot of quality by pH category
plt.figure(figsize=(10, 6))
quality_by_ph.plot(kind='bar', color='salmon')
plt.title('Average Wine Quality by pH Level', fontsize=14)
plt.xlabel('pH Category', fontsize=12)
plt.ylabel('Average Quality', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Analyze other attributes by pH category
attributes_by_ph = wine_df.groupby('pH_category').agg({
    'alcohol': 'mean',
    'volatile acidity': 'mean',
    'residual sugar': 'mean',
    'sulphates': 'mean',
    'quality': ['mean', 'max', 'min', 'std']
}).round(2)

print("\nKey attributes by pH category:")
print(attributes_by_ph)

# Find which subset has highest density and explain why
density_by_ph = wine_df.groupby('pH_category')['density'].mean().sort_values(ascending=False)
print("\nMean density by pH category (highest to lowest):")
print(density_by_ph)

highest_density_category = density_by_ph.index[0]
print(f"\nThe pH category with highest density is: {highest_density_category}")

# Investigate why this subset has higher density by comparing other attributes
high_density_analysis = wine_df.groupby('pH_category').agg({
    'residual sugar': 'mean',
    'alcohol': 'mean',
    'fixed acidity': 'mean'
}).loc[highest_density_category]

print("\nKey factors affecting density in the highest density pH category:")
print(high_density_analysis)

print("\nExplanation: The subset with highest density likely has this characteristic because of:")
print("1. Higher residual sugar content (sugar increases density)")
print("2. Potentially lower alcohol content (alcohol decreases density)")
print("3. Differences in acid composition (acids can affect density)")

## 10. Correlation Matrix and Heatmap

### Making a correlation matrix with the clean wine data

In [ ]:
wine_df_clean

In [ ]:
matrix = wine_df_clean.corr()

In [ ]:
print("Correlation matrix is : ")
print(matrix)

### Making a heatmap based on the matrix with all cleaned wine data

In [ ]:
sbn.heatmap(matrix, cmap='plasma', annot=True, fmt=".1f")

##### We can see here that alcohol clearly stands out with a correlation of 0.5, followed by type and citric acid - both at 0.1. On the other side of the spectrum we have density and volatile acidity which share a correlation -0.3, hinting that a higher content of those two might influence the quality of the wine negatively.

### Making a bar chart to clarify which wine characteristic has the highest correlation with the wine's quality. (excluding quality of course)

In [ ]:
quality_correlation = matrix['quality'].drop('quality').sort_values(ascending=False)

In [ ]:
print(quality_correlation)

In [ ]:
plt = quality_correlation.plot(kind='bar')

##### So, as this bar chart clearly pictures, the alcohol content has the biggest correlation with the wine's quality. The higher a wine's alcohol content - the higher its quality generally is. Secondly the winetype has the second highest correlation and citric acid third. Density has the lowest correlation, so the higher the density in the wine - the lower its quality may be. The same applies to the other negative correlation values: total sulfur dioxide, residual sugar, fixed acidity, chlorides and volatile acidity

### Delving deeper into the most noteworthy correlation values:

#### Alcohol and quality

In [ ]:
alc_to_q = wine_df_clean.groupby('alcohol')['quality'].mean()

In [ ]:
alc_to_q.plot.area(xlabel='Alcohol content in %', ylabel='Average Quality')

##### There is a general tendency that the higher the alcohol content - the higher the wine's quality. But once the alcohol content is 15% or higher, the quality falls off a cliff - so too high alcohol content isn't great either.

#### Type of wine and correlation with quality

In [ ]:
type_to_q = wine_df_clean.groupby('type')['quality'].mean()
print(f"Average quality for red wine (0): {type_to_q[0]}")
print(f"Average quality for white wine (1): {type_to_q[1]}")

##### So this means that if the wine is a white wine it typically corresponds to a higher quality wine compared to if it was a red wine

#### Citric acid and correlation with quality

In [ ]:
cit_acid_to_q = wine_df_clean.groupby('citric acid')['quality'].mean()
cit_acid_to_q.plot.area(xlabel='Citric acid content', ylabel='Average Quality')

##### Overall the quality generally increases with higher citric acid content, but interestingly, there is a pretty big spike at around 0.75 in citric acid, where the quality increases drastically. So this is the best amount of citric acid to have in a wine - compared to the steep fall in quality by around 0.90 in citric acid content, which we should avoid.

#### Density of the wine and correlation with quality

In [ ]:
density_to_q = wine_df_clean.groupby('density')['quality'].mean()
density_to_q.plot.area(xlabel='The density of the wine', ylabel='Average Quality')

##### Generally the lower the density of the wine - the higher quality the wine tends to be, and the quality completely stagnates after the 1.005 mark in density.

#### Volatile acidity and correlation with quality

In [ ]:
vol_acid_to_q = wine_df_clean.groupby('volatile acidity')['quality'].mean()
vol_acid_to_q.plot.area(xlabel='Volatile acid content', ylabel='Average Quality')

##### Volatile acid content is very interesting, as quality starts out high with a low volatile acidity - and the quality gets progressively lower as the acidity increases. It jumps up and down in quality quite suddenly between the values of 1.0 and 1.4 in volatile acidity. Maybe it is some kind of chemistry between the wine and the volatile acidity that just doesn't work out well at certain values - maybe it creates an imbalance when comparing to the other characteristics of the wine? In any case, the wine quality continuously decreases when volatile acidity is above 1.4. So all in all the general tendency is the lower the volatile acidity - the higher the wine quality.

#### Figuring out the lowest correlation with quality

In [ ]:
ph_to_q = wine_df_clean.groupby('pH')['quality'].mean()
ph_to_q.plot.area(xlabel='pH value', ylabel='Average Quality')

##### As seen when calculating quality_correlation - pH is the value closest to 0 at 0.039733 - therefore pH is the value that has the least impact on the wine's quality. As pictured on the graph the tendency of the plot is generelly quite stagnant even as the quality and the pH value increases. There's some exceptions to this though, with spikes in quality in between, but the general consensus is that there is not much correlation between the pH value of the wine's quality compared to the other wine characteristics. Though it seems that there might be a correlation between a pH value higher than 3.9 and a higher quality wine.

### Data preparation for futher analysis

### 11. Identify residual sugar outliers, exact rows and remove said rows
Explore the feature ‘residual sugar’. Does it contain outliers? On which rows of the data 
frame are they found? Remove those rows

##### Examining features earlier already revealed outliers, and task 7 showed outliers visually in diagrams. The method of finding outliers with IQR, allows us to remove those that fall outside the "whiskers". Like in task 7, we've cleaned red and white wines seperately. This should ensure we find outliers correctly by taking into account the differences between the two types of wine. Cleaning the combined dataset could risk outliers not being found, or rows mistakenly being flagged as outliers.

In [ ]:
red_wine = wine_df_clean[wine_df_clean['type'] == 0]
all_red_outliers = dp.iqr_outliers(red_wine, 'residual sugar')
white_wine = wine_df_clean[wine_df_clean['type'] == 1]
all_white_outliers = dp.iqr_outliers(white_wine, 'residual sugar')

In [ ]:
all_iqr_outliers = pd.concat([all_red_outliers, all_white_outliers])
print(all_iqr_outliers)

In [ ]:
print(all_iqr_outliers.index.tolist())

##### We now know all the indexes of all rows that are outliers

In [ ]:
# print a specific row or multiple
print(wine_df_clean.loc[9])

In [ ]:
wine_df_no_iqr_outliers = wine_df_clean.drop(index=all_iqr_outliers.index)

In [ ]:
wine_df_no_iqr_outliers

##### Now all outlier rows have been removed with "wine_df_no_iqr_outliers" being our outlier-free dataframe

### 12. Remove the attributes, which aren’t correlated with the wine quality, as well as the attributes that are highly correlated with another independent attribute.

#### Least correlation with wine quality

In [ ]:
quality_correlation
print(quality_correlation.abs().sort_values().head(5))

##### So we can see that these 5 have the lowest correlation with the quality with the wine, and especially pH and sulphates have very little correlation, as they round down to 0.0 - as seen on the heatmap below. So we want to remove those two as they don't affect the quality values. 

In [ ]:
sbn.heatmap(matrix, cmap='spring', annot=True, fmt=".1f")

#### High correlation and dependency with another independent attribute

In [ ]:
abs_corr = matrix.copy().abs()
np.fill_diagonal(abs_corr.values, np.nan)
corr_pairs = abs_corr.unstack()

# Dropping duplicate pairs - so we fx don't compare citric acid with itself
corr_pairs = corr_pairs.dropna().sort_values(ascending=False)
corr_pairs = corr_pairs[corr_pairs.index.get_level_values(0) != corr_pairs.index.get_level_values(1)]
corr_pairs = corr_pairs[~corr_pairs.duplicated()]


print("Top 5 most strongly correlated wine characteristic pairs (both positively and negatively correlated):")
print(corr_pairs.head(5))

##### As calculated we can see that the correlation between total sulfur dioxide and both free sulfur dioxide and type are high, so it seems that the value of total sulfur dioxide is dependant on those two. Therefore we want to drop total sulfur dioxide as it doesn't seem to function independently.

#### Removing the columns: pH, sulphates and total sulfur dioxide

In [ ]:
wine_df_no_iqr_outliers = wine_df_no_iqr_outliers.drop(columns=['pH', 'sulphates', 'total sulfur dioxide'])

In [ ]:
wine_df_no_iqr_outliers

In [ ]:
wine_df_no_iqr_outliers.shape

##### The columns have been successfully removed because we had 13 columns before :)

### 13. Transform the data by applying PCA (Principle Component Analysis).

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Select only numerical features for PCA
numeric_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
                  'chlorides', 'free sulfur dioxide', 'density', 'alcohol']

X = wine_df_no_iqr_outliers[numeric_columns]
print("Shape of data for PCA:", X.shape)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA()
pca_result = pca.fit_transform(X_scaled)

# Calculate variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Print variance for each component
print("variance ratio by component")
for i, var in enumerate(explained_variance):
    print(f"PC{i+1}: {var:.4f} ({cumulative_variance[i]:.4f} cumulative)")

# Determine number of components needed to explain 80% of variance
n_components = np.argmax(cumulative_variance >= 0.8) + 1
print(f"\nNumber of principal components needed to explain 80% of variance: {n_components}")

In [ ]:
# Analyze feature contributions to principal components
loadings = pd.DataFrame(
    pca.components_.T, 
    columns=[f'PC{i+1}' for i in range(len(numeric_columns))],
    index=numeric_columns
)

# Show loadings for the first 3 principal components
print("Feature loadings for the first 3 principal components:")
print(loadings.iloc[:, :3].round(3))

# Visualize loadings for first two components
plt.figure(figsize=(12, 8))
loadings_pc1 = loadings['PC1'].sort_values()
loadings_pc1.plot(kind='barh', color='blue', alpha=0.6)
plt.title('Feature Contributions to PC1')
plt.xlabel('Loading Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 8))
loadings_pc2 = loadings['PC2'].sort_values()
loadings_pc2.plot(kind='barh', color='green', alpha=0.6)
plt.title('Feature Contributions to PC2')
plt.xlabel('Loading Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze correlations between principal components and quality
pc_quality_df = pd.DataFrame({
    'quality': wine_df_no_iqr_outliers['quality']
})

# Add the principal components to the dataframe
for i in range(n_components):
    pc_quality_df[f'PC{i+1}'] = pca_result[:, i]

# Calculate correlations with quality
pc_quality_corr = pc_quality_df.corr()['quality'].drop('quality')
print("Correlation between principal components and wine quality:")
print(pc_quality_corr.sort_values(ascending=False))

# Visualize correlations
plt.figure(figsize=(10, 6))
pc_quality_corr.sort_values().plot(kind='barh', color='purple', alpha=0.6)
plt.title('Correlation of Principal Components with Wine Quality')
plt.xlabel('Correlation Coefficient')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 14. Print sample from final dataset
Print out ten random rows from the final dataset as a prove of concept

In [ ]:
wine_df_no_iqr_outliers.sample(10)

##### 10 random rows. Free of duplicates, missing values and outliers, and with pH, sulphates and total sulfur dioxide columns also removed from earlier.

### 15. 
Use Streamlit to build an application, which allows interactive data loading and visualization
of the exploratory analysis. Apply both 2D and 3D data visualization techniques.

##### Unsure of how to do this, so it was skipped for now

### 16.
The quality of wine is a complex category that also depends on multiple non-numeric 
parameters, such as geographical origin, the production technology, and human taste. 
Extend your application with useful public information, related to the quality of wine. Use
either the web, Wikipedia, or Youtube as a source. Implement LLM for text summarization, if 
necessary

##### Image reader for wines

In [ ]:
file1 ="wine1.jpg"
path1=os.path.join(media_path, file1)
display(Image(filename=path1, width=400))

In [ ]:
result=explain(path1)
display(Markdown(result))